# 1. Plik konfiguracyjny

tworzymy plik **.env** i wpisujemy np.:

- pip install python-dotenv

- plik moe zawierać np. takie informacje:

FIRST_NAME = Marek \
LAST_NAME = Budzicz \
EMAIL = markowian22@gmail.com 

In [101]:
from os import environ, getenv
from dotenv import load_dotenv
s

In [103]:
load_dotenv()
print(f"'FIRST_NAME' : {getenv('FIRST_NAME')}")
print(f"'LAST_NAME' : {getenv('LAST_NAME')}")
print(f"'EMAIL' : {getenv('EMAIL')}")

'FIRST_NAME' : MAREK
'LAST_NAME' : BUDZICZ
'EMAIL' : markowian22@gmail.com


# 2. Contex Manager

In [ ]:
with open('test.txt', 'r') as file:
    lines = file.readlines()
    print(lines)

# 3. Obsługa baz danych

In [ ]:
import sqlite3

with sqlite3.connect('baza.db') as connection:
    cursor = connection.cursor()
    cursor.execute('SELECT * from books')
    print(cursor.fetchall())

    for book in cursor.fetchall():
        print(book)

In [ ]:
cursor.execute('SELECT * FROM books WHERE author=?' , ('Marek Budzicz', )) # przykład w którym dodajemy dane z pythona w tupli do zapytania
cursor.execute('INSERT INTO items VALUES(null, ?, ?, ?, ?)')  # przykład w którym dodajemy wiele elemenów pod kazda kolumnę

### Testowanie połączenia z bazą danych

In [ ]:
## plik który nazywa sie data_base.py
## ta część kodu to połączenie się z bazą danych oraz wywołanie zapytania

def create_connection():
    with sqlite3.connect('baza.db') as connection:
        cursor = connection.cursor()
        return cursor
    
def get_data(cursor):
    data = []
    cursor.execute('SELECT * FROM books WHERE author=?' , ('Marek Budzicz', ))

    for book in cursor.fetchall():
        data.append(book)
    return data

if __name__=='__main__':
    cursor = create_connection()
    data = get_data(cursor)
    print(data)

##### testowanie połączenia

In [25]:
import pytest
import sqlite3
from data_base.py import get_data ## tutaj łączymy się z kodem napisanym wyzej

@pytest.fixture
def create_database():
    connection = sqlite3.connect(':memory') ## tutaj tworzymy pamięć w której będziemy testowac naszą baze danych, zeby nie korzystac z glownej bazy tylko w formie testu z bazy w pamieci
    cursor = connection.cursor()
    cursor.execute('''
        CREATE TABLE books
        (id INTEGER, title TEXT, author TEXT, created_at DATE)
    ''')
    data_sample = [
        (1, 'odnoga', 'Marus', '2020-12-23'),
        (2, 'marek koks', 'Jula', '2024-02-02')
        ]

    cursor.executemany('INSERT INTO books VALUES(?, ?, ?, ?)', data_sample)
    return cursor

def test_get_data(create_database):
    cursor = create_database
    print(cursor)
    assert data[0] ==   (1, 'marek', 'koks', '2022-12-12') # tutaj dajemy wynik jaki nas interesuje zeby sprawdzic np. 



# 4. Wysylka maila

- biblioteka **smptlib**

In [ ]:
import smtplib, ssl

# zmienne do SMTP_SSL 
smtp_server = "smtp.gmail.com"
port = 465

# zrobiłem funkcje zeby móc testowac plik
def send_mail():
    sender_email = "budziczmarek@gmail.com"
    password = 'peye wzri mhdz kryh'
    to_person = 'markowian22@gmail.com'

    msg = 'hej'

    context = ssl.create_default_context()
    context.check_hostname = False
    context.verify_mode = ssl.CERT_NONE
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server: # tworzymy server 
        server.login(sender_email, password) # logowanie się do maila
        server.sendmail(sender_email, to_person, 'co tam marus')
        




## testowanie wysyłki maila

In [71]:
# importujemy plik powyzej
from unittest.mock import patch # importujemy mocka 

@patch('smtplib.SMTP_SSL')
def test_send_mail(mock_smtp):
    send_mail()
    # tu sprawdzamy ze smtplib został wywołany
    mock_smtp.assert_called() 
    # tu sprawdzamy czy zostaliśmy zalogowani
    contex = mock_smtp.return_value.__enter__.return_value
    contex.login.assert_called()
    # tu sprawdzamy czy mail został wysłany
    contex.sendmail.assert_called_with('budziczmarek@gmail.com', 'markowian22@gmail.com', 'treść maila')

# w terminal wpisujemy taka komende aby stestowac plik 
# python -m pytest send_mail.py 

# 5. Tuple nazwane

korzystamy z tego ze jak chcemy stworzyc jakas mala klaske ktróra trzyma tylko imie i nazwisko to zamiast 
tworzyc klase to tworzymy tuple nazwana. Jest ona niemodyfikowalana i jest przyjemna dla oka

In [90]:
from collections import namedtuple # importujemy tuple nazwaną

User = namedtuple('User', ['name', 'surname', 'age'])
adam = User('Adam', 'Budzicz', '26')

print(adam.age)
print(adam.name)
print(adam.surname)


26
Adam
Budzicz


# 6. Szablony zwane TEMPLATEAMI

In [93]:
from string import Template

message = Template('$first_name zajdł dziś $count pączków!')
info = [
    ('marek', 10),
    ('karol', 20)
]

for first_name, count in info:
    text = message.substitute(first_name=first_name, count=count)
    print(text)

marek zajdł dziś 10 pączków!
karol zajdł dziś 20 pączków!
